## Advanced Lane Finding Project (for Video)

The goals / steps of this project are the following:

* Uses camera calibration matrix and distortion coefficients
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Make a sanity check to fitted lines
* Output a video to display of the lane boundaries and numerical estimation of lane curvature and vehicle position
* Output a video to display also the warped image, the thresholded image and the sanity check values for tunning purpose

In [1]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
import pickle
import matplotlib.image as mpimg

%matplotlib qt
%matplotlib inline

Based on **P2_Images**, I would join functions into a Class, called Line, to be able to used functions on a video.

## New functions are created:
### 1. record_functioname: 
I split the video on 4 parts, in order to show on video also the warped image, the thresholded image and the sanity check values. This video is done for tunning purpose. To record this video, I extend many existend functions to be able to return a color image with relevant information. This function name will always start with **record**

### 2. sanity_check: 

This function check the plausibility of the fitted line using the following approaches:
#### 2.1. variation of the curvature of the line: 
2.1.1. if curvature is up to 6000m, an acceptable variation is 30%
2.1.2. if curvature is higher than 6000m, I assume almost straight line (curvature = infinite), which causes high oscillation on the values, eventhough line is rightfully recognized. However, if oscillation is 1000% more, I would suspect on wrong line recognzition and avoid this frame

#### 2.2. both lines should be separated by almost the same distance: 
by looking of the distance between the first and last point of the line

#### 2.3. both lines should be almost parallel: 
by looking of the distance between the first and middle point of the line

#### 2.4. both lines should have the same curvature direction: 
by calculating the **m** of each line. This propose is only used if line is not straight (curvature < 6000m)

#### 2.5. first points of lines should be plausible: 
The first point of the line can not change abruptly

## All Functions:

In [2]:
# function, which undistort picture (take out calibration error from pictures)
def cal_undistort(img, mtx, dist):
    undist = cv2.undistort(img, mtx, dist, None, mtx)
    return undist

In [3]:
# Thresholded Image, using Sobel X and R Channel 
def R_sobelx(img, sx_thresh=(30, 100)):
    
    # Get R Layer
    R_channel = img[:,:,0]

    # Sobel x, using l_channel
    sobelx = cv2.Sobel(R_channel, cv2.CV_64F, 1, 0) # Take the derivative in x
    abs_sobelx = np.absolute(sobelx) # Absolute x derivative to accentuate lines away from horizontal
    scaled_sobel = np.uint8(255*abs_sobelx/np.max(abs_sobelx))
    
    # Threshold x gradient
    sxbinary = np.zeros_like(scaled_sobel)
    sxbinary[(scaled_sobel >= sx_thresh[0]) & (scaled_sobel <= sx_thresh[1])] = 1
    
    return sxbinary

In [4]:
def RH_sobelx(img, sx_thresh=(30, 100), sh_thresh = (30,90)):
    
    img_copy = np.copy(img)
    hls = cv2.cvtColor(img_copy, cv2.COLOR_RGB2HLS)
    H_channel = hls[:,:,0]
    
    # Get R G B Layer
    R_channel = img[:,:,0]
    #R_channel[(R_channel <= 50)] = 255
    # Sobel x, using l_channel
    sobelx = cv2.Sobel(R_channel, cv2.CV_64F, 1, 0) # Take the derivative in x
    abs_sobelx = np.absolute(sobelx) # Absolute x derivative to accentuate lines away from horizontal
    scaled_sobel = np.uint8(255*abs_sobelx/np.max(abs_sobelx))
    
    # Threshold x gradient
    sxbinary = np.zeros_like(scaled_sobel)
    sxbinary[(scaled_sobel >= sx_thresh[0]) & (scaled_sobel <= sx_thresh[1])] = 1
    
    #threshold S channel
    #s_binary = np.zeros_like(s_channel)
    sxbinary[(H_channel <= sh_thresh[1]) & (H_channel >= sh_thresh[0])] = 0
    
    return sxbinary

In [5]:
# Thresholded Image, usind S Channel and minus R Channel
def SR_thresholded(img, s_thresh=(150, 255), sr_thresh = 30):
    # Threshold color channel
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    s_channel = hls[:,:,2]
    
    # Threshold S Channel
    s_binary = np.zeros_like(s_channel)
    s_binary[(s_channel >= s_thresh[0]) & (s_channel <= s_thresh[1])] = 1
    
    # Removing Pixels from S Channel using R Channel 
    R_channel = img[:,:,0]
    s_binary[(R_channel <= sr_thresh)] = 0
    
    return s_binary

In [6]:
def SRH_thresholded(img, s_thresh=(150, 255), sr_thresh = 30, sh_thresh = (28,90)): #(50,90)
    # Threshold color channel
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    s_channel = hls[:,:,2]
    H_channel = hls[:,:,0]
    
    s_binary = np.zeros_like(s_channel)
    s_binary[(s_channel >= s_thresh[0]) & (s_channel <= s_thresh[1])] = 1
    R_channel = img[:,:,0]
    s_binary[(R_channel <= sr_thresh)] = 0
    
    s_binary[(H_channel <= sh_thresh[1]) & (H_channel >= sh_thresh[0])] = 0
    
    return s_binary

In [7]:
# Edit this function to create your own pipeline.
def pipeline(img, s_thresh=(90, 255), sx_thresh=(20, 100)):
    img = np.copy(img)
  
    sxbinary = RH_sobelx(img, sx_thresh=sx_thresh)
    s_binary = SRH_thresholded(img, s_thresh=s_thresh)
       
    # combine channel
    img_binary =  np.zeros_like(s_binary)
    img_binary = np.bitwise_or(s_binary,sxbinary)

    return img_binary

In [8]:
# function for binary threshold
def pipeline_color(img, s_thresh=(90, 255), sx_thresh=(20, 100)):
    img = np.copy(img)
  
    sxbinary = RH_sobelx(img, sx_thresh=sx_thresh)
    s_binary = SRH_thresholded(img, s_thresh=s_thresh)
         
    # Stack each channel (different colors)
    color_binary = np.dstack(( np.zeros_like(sxbinary), sxbinary, s_binary)) * 255

    
    cv2.putText(color_binary, text='Blue comes from S-Channel, threshold =  ' + str(s_thresh), org=(50,50),
                    fontFace= cv2.FONT_HERSHEY_SIMPLEX, fontScale=1, color=(255,255,255),
                    thickness=2, lineType=cv2.LINE_AA)
    
    cv2.putText(color_binary, text='Green comes from R-Channel and Sobel X, threshold = ' + str(sx_thresh), org=(50,100),
                    fontFace= cv2.FONT_HERSHEY_SIMPLEX, fontScale=1, color=(255,255,255),
                    thickness=2, lineType=cv2.LINE_AA)
    
    return color_binary

In [15]:
# Define a class to receive the characteristics of each line detection
class Line():
    
    # Read in the saved camera matrix and distortion coefficients
    dist_pickle = pickle.load( open( "wide_dist_pickle.p", "rb" ) )
    mtx = dist_pickle["mtx"]
    dist = dist_pickle["dist"]
    
    def __init__(self):
        # check of initialization
        self.count=0
        self.configurated =  False
        # matrix values
        self.perspective_M = None
        self.Minv = None
        
        # y values of the line on bird view
        self.ploty =  None
        # x values of the line on bird view
        self.left_fitx= None
        self.right_fitx = None
        # parameters of the line on bird view
        self.left_fit= None
        self.right_fit= None
        
        # x values of the line on bird view
        self.old_left_fitx= None
        self.old_right_fitx = None
        # parameters of the line on bird view
        self.old_left_fit= None
        self.old_right_fit= None
        
        # image size
        self.img_size = None
        # parameter to change to meters
        self.ym_per_pix = None
        self.xm_per_pix = None
        
        # was the line detected in the last iteration?
        self.detected = False
        self.count_undetected = 0
        
        
        # x values of the last n fits of the line
        self.recent_xfitted = [] 
        #average x values of the fitted line over the last n iterations
        self.bestx = None     
        #polynomial coefficients averaged over the last n iterations
        self.best_fit = None  
        #difference in fit coefficients between last and new fits
        self.diffs = np.array([0,0,0], dtype='float') 
        #x values for detected line pixels
        self.allx = None  
        #y values for detected line pixels
        self.ally = None  
        
    def __call__(self, img):
    # any process you want
        self.count += 1
        #undistort
        img_undist = cal_undistort(img, Line.mtx, Line.dist)
        #binary thershold
        img_binary = pipeline(img_undist)
        #color_binary = pipeline_color(img_undist)

        #starting configuration
        if (not self.configurated):
             #bird-eye-view
            self.perspective_M = self.bird_eye_view_matrix(img_binary)
            self.Minv = np.linalg.inv(self.perspective_M)
            binary_warped = self.bird_eye_view_transformation(img_binary)
            
            self.img_size = (binary_warped.shape[1], binary_warped.shape[0])
            self.ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0])
            #ploty = np.linspace(0, img_shape[0]-1, img_shape[0])
            # variables
            offset = 300
            self.ym_per_pix = 50/self.img_size[1]
            self.xm_per_pix = 3.7/(self.img_size[0]-2*offset)
            self.configurated = True

        else:
            self.old_left_fitx = self.left_fitx
            self.old_right_fitx = self.right_fitx
            self.old_left_fit = self.left_fit
            self.old_right_fit = self.right_fit
            
            binary_warped = self.bird_eye_view_transformation(img_binary)
              
        #First Frame- Complete Search:        
        if (not self.detected):
            #print("First Frame")
            # Here left_fitx ad left_fit (also right) are calculated
            self.fit_polynomial(binary_warped)
            #self.search_around_poly(binary_warped)
            self.detected = True
            left_curverad, right_curverad = self.measure_curvature_real()
            self.old_left_fitx = self.left_fitx
            self.old_right_fitx = self.right_fitx
            self.old_left_fit = self.left_fit
            self.old_right_fit = self.right_fit
            
            #if First Frame is wrong, ignore this Frame and use the next
            margin=150
            check, [var_radius, var_topbottom, var_middlebottom, var_bottom_right, var_bottom_left] = self.sanity_check(left_curverad, right_curverad, margin)
            if (not check):
                self.detected = False
                
                #color_warped = self.display_sanity_check(color_warped, var_radius, var_topbottom, var_middlebottom,  var_bottom_right, var_bottom_left)
                #result4 = self.display_4img(img, color_warped, img_binary, color_binary)
                return img
            else:
                #invert transform
                #color_warped = self.display_sanity_check(color_warped, var_radius, var_topbottom, var_middlebottom,  var_bottom_right, var_bottom_left)
                result= self.invert_transformation(binary_warped, img_undist)
                result = self.show_curverad_position(result, left_curverad, right_curverad)
                #result4 = self.display_4img(result, color_warped, img_binary, color_binary)
                return result
            
        #Complete Search  
        elif self.count_undetected > 5:
            self.fit_polynomial(binary_warped)
            #print(self.left_fit)
            #print(self.right_fit)
            #self.search_around_poly(binary_warped)           
            left_curverad, right_curverad = self.measure_curvature_real()
            
            margin=150
            check, [var_radius, var_topbottom, var_middlebottom, var_bottom_right, var_bottom_left] = self.sanity_check(left_curverad, right_curverad, margin)
            #not good: use old values
            if (not check):
                self.left_fitx = self.old_left_fitx
                self.right_fitx = self.old_right_fitx
                self.left_fit = self.old_left_fit
                self.right_fit = self.old_right_fit
                left_curverad, right_curverad = self.measure_curvature_real()
                #print("Sanity Change!")
                self.count_undetected += 1
                
                result= self.invert_transformation(binary_warped, img_undist)
                cv2.putText(result, text='Sanity check not passed on Window Search!', org=(50,250),
                    fontFace= cv2.FONT_HERSHEY_SIMPLEX, fontScale=1, color=(0,255,0),
                    thickness=2, lineType=cv2.LINE_AA)
                
                #visualization
                #color_warped = cv2.polylines(color_warped, np.array([np.transpose(np.vstack([self.left_fitx, self.ploty]))], np.int32), False, (255,255,255), 4)
                #color_warped = cv2.polylines(color_warped, np.array([np.transpose(np.vstack([self.right_fitx, self.ploty]))], np.int32), False, (255,255,255), 4)
                
            #good: use this values
            else:
                self.count_undetected = 0
                result= self.invert_transformation(binary_warped, img_undist)
        
            #invert transform
            result= self.invert_transformation(binary_warped, img_undist)
            #color_warped = self.display_sanity_check(color_warped, var_radius, var_topbottom, var_middlebottom, var_bottom_right, var_bottom_left)
            result = self.show_curverad_position(result, left_curverad, right_curverad)
            #result4 = self.display_4img(result, color_warped, img_binary, color_binary)
            return result
        
        #Simple Search
        else:
           # print("X Frame")
            self.search_around_poly(binary_warped)
            #print(self.left_fit)
            #print(self.right_fit)
            # Calculate the radius of curvature in meters for both lane lines
            left_curverad, right_curverad = self.measure_curvature_real()
        
            margin=10
            check, [var_radius, var_topbottom, var_middlebottom,  var_bottom_right, var_bottom_left] = self.sanity_check(left_curverad, right_curverad, margin)
            
            if (not check):
                self.left_fitx = self.old_left_fitx
                self.right_fitx = self.old_right_fitx
                self.left_fit = self.old_left_fit
                self.right_fit = self.old_right_fit
                left_curverad, right_curverad = self.measure_curvature_real()
                #print("Sanity Change!")
                self.count_undetected+= 1
                
                result= self.invert_transformation(binary_warped, img_undist)
                cv2.putText(result, text='Sanity check not passed!', org=(50,250),
                    fontFace= cv2.FONT_HERSHEY_SIMPLEX, fontScale=1, color=(0,255,0),
                    thickness=2, lineType=cv2.LINE_AA)
                
                #visualization:
                #color_warped = cv2.polylines(color_warped, np.array([np.transpose(np.vstack([self.left_fitx, self.ploty]))], np.int32), False, (255,255,255), 4)
                #color_warped = cv2.polylines(color_warped, np.array([np.transpose(np.vstack([self.right_fitx, self.ploty]))], np.int32), False, (255,255,255), 4)
                
            else:
                self.count_undetected = 0
                result= self.invert_transformation(binary_warped, img_undist)
        
            #invert transform
            #result= self.invert_transformation(binary_warped, img_undist)
            #color_warped = self.display_sanity_check(color_warped, var_radius, var_topbottom, var_middlebottom, var_bottom_right, var_bottom_left)
            result = self.show_curverad_position(result, left_curverad, right_curverad)
            #result4 = self.display_4img(result, color_warped, img_binary, color_binary)
            return result
    
    
    # plausibility of the lines check function
    def sanity_check(self, left_curverad, right_curverad, margin):
        check = True
        # Checking that they have similar curvature
    
        mean_radius = (left_curverad + right_curverad)/2
        var_radius = round(abs((left_curverad - right_curverad)/left_curverad),3)
        #mean_radius =  6000 is straight line and stright line are random values (infinite)
        if (mean_radius <6000) &  (var_radius > 0.3): #0.6 1
            check = False
            
        if (mean_radius >=6000) &  (var_radius > 10):
            check = False
        
        #Checking that they are separated by approximately the right distance horizontally
        top_distance = self.right_fitx[-1] - self.left_fitx[-1]
        bottom_distance = self.right_fitx[0] - self.left_fitx[0]
        var_topbottom=round(abs((top_distance - bottom_distance)/top_distance),3)
        if  var_topbottom > 0.6: #0.3 #0.1 0.2 
            check = False
            
       #Checking that they are roughly parallel
        middle=int(self.right_fitx.size/2)
        middle_distance = self.right_fitx[middle] - self.left_fitx[middle]
        var_middlebottom =round(abs((top_distance - middle_distance)/top_distance),3)
        #bottom_distance = self.right_fitx[-1] - self.left_fitx[-1]
        if  var_middlebottom > 0.25: #0.15 #0.1 0.2
            check = False 
            
        #Check curvature direction
        sign_right = np.sign(self.right_fitx[middle] - self.right_fitx[0])
        sign_left = np.sign(self.left_fitx[middle] - self.left_fitx[0])
        if (mean_radius <6000) & (sign_right != sign_left):
            check = False 
            
        #Reliable bottom position
        #margin = 150
        var_bottom_right =  round(abs((self.right_fitx[-1] - self.old_right_fitx[-1])),1)
        var_bottom_left =  round(abs((self.left_fitx[-1] - self.old_left_fitx[-1])),1)
        if (var_bottom_left > margin) or (var_bottom_right > margin) :
            check = False
            
        return check, [var_radius, var_topbottom, var_middlebottom, var_bottom_right, var_bottom_left ]
        
    # display important information on the image     
    def show_curverad_position(self, img, left_curverad, right_curverad):
        #left_curverad, right_curverad = self.measure_curvature_real()
        curverad = int((left_curverad + right_curverad)/2)

        position= self.measure_vehicle_position()

        cv2.putText(img, text='Radius of Curvature = ' + str(curverad) + ' m', org=(50,50),
                    fontFace= cv2.FONT_HERSHEY_SIMPLEX, fontScale=1, color=(0,255,0),
                    thickness=2, lineType=cv2.LINE_AA)
        if position > 0:
            cv2.putText(img, text='Position of Vehicle = ' + str(abs(position)) + ' m right of center', org=(50,100),
                    fontFace= cv2.FONT_HERSHEY_SIMPLEX, fontScale=1, color=(0,255,0),
                    thickness=2, lineType=cv2.LINE_AA)
        elif position < 0:
            cv2.putText(img, text='Position of Vehicle = ' + str(abs(position)) + ' m left of center', org=(50,100),
                    fontFace= cv2.FONT_HERSHEY_SIMPLEX, fontScale=1, color=(0,255,0),
                    thickness=2, lineType=cv2.LINE_AA)
        else:
            cv2.putText(img, text='Vehicle is on the middle!', org=(50,100),
                    fontFace= cv2.FONT_HERSHEY_SIMPLEX, fontScale=1, color=(0,255,0),
                    thickness=2, lineType=cv2.LINE_AA)
            
        cv2.putText(img, text='Frame = ' + str(self.count), org=(50,150),
                    fontFace= cv2.FONT_HERSHEY_SIMPLEX, fontScale=1, color=(0,255,0),
                    thickness=2, lineType=cv2.LINE_AA)
        
        return img
    
    #calculate vehicle position
    def measure_vehicle_position(self):
        center_car = self.img_size[0]/2

        # Point Format: np.array([[[0, 0]]], dtype=np.float32)
        point_warped_right = np.float32([[[self.right_fitx[-1], self.img_size[1]]]])
        point_warped_left = np.float32([[[self.left_fitx[-1], self.img_size[1]]]])
        point_unwarped_right = cv2.perspectiveTransform(point_warped_right, self.Minv)[0][0][0]
        point_unwarped_left = cv2.perspectiveTransform(point_warped_left, self.Minv)[0][0][0]

        center_road = (point_unwarped_right - point_unwarped_left)/2 + point_unwarped_left
        position= round((center_car - center_road)*self.xm_per_pix,2)

        return position
    
    # calculate curvature
    def measure_curvature_real(self):
        '''
        Calculates the curvature of polynomial functions in meters.
        '''
        # Define y-value where we want radius of curvature
        # We'll choose the maximum y-value, corresponding to the bottom of the image
        y_eval = np.max(self.ploty)

        left_fit_cr = [(self.left_fit[0]*self.xm_per_pix) / (self.ym_per_pix**2), (self.xm_per_pix/self.ym_per_pix)*self.left_fit[1], self.left_fit[2]]
        right_fit_cr = [(self.right_fit[0]*self.xm_per_pix) / (self.ym_per_pix**2), (self.xm_per_pix/self.ym_per_pix)*self.right_fit[1], self.right_fit[2]]

        ##### TO-DO: Implement the calculation of R_curve (radius of curvature) #####
        left_curverad = ((1+(left_fit_cr[0]*2*y_eval+left_fit_cr[1])**2)**(1.5))/(np.abs(2*left_fit_cr[0]))  ## Implement the calculation of the left line here
        right_curverad = ((1+(right_fit_cr[0]*2*y_eval+right_fit_cr[1])**2)**(1.5))/(np.abs(2*right_fit_cr[0]))  ## Implement the calculation of the right line here

        return left_curverad, right_curverad
    
    # invert transformation
    def invert_transformation(self, binary_warped, image_undist):
        # Create an image to draw the lines on
        warp_zero = np.zeros_like(binary_warped).astype(np.uint8)
        color_warp = np.dstack((warp_zero, warp_zero, warp_zero))

        # Recast the x and y points into usable format for cv2.fillPoly()
        pts_left = np.array([np.transpose(np.vstack([self.left_fitx, self.ploty]))])
        pts_right = np.array([np.flipud(np.transpose(np.vstack([self.right_fitx, self.ploty])))])
        pts = np.hstack((pts_left, pts_right))

        # Draw the lane onto the warped blank image
        cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))

        # Warp the blank back to original image space using inverse perspective matrix (Minv)
        newwarp = cv2.warpPerspective(color_warp, self.Minv, (image_undist.shape[1], image_undist.shape[0])) 
        # Combine the result with the original image
        result = cv2.addWeighted(image_undist, 1, newwarp, 0.3, 0)

        return result
    
    ####### Line Finding Functions ###################### 
    # ***** Simple Search *****
    def fit_poly(self, leftx, lefty, rightx, righty):
        ### TO-DO: Fit a second order polynomial to each with np.polyfit() ###
        self.left_fit = np.polyfit(lefty, leftx, 2)
        self.right_fit = np.polyfit(righty, rightx, 2)
        # Generate x and y values for plotting
        #ploty = np.linspace(0, img_shape[0]-1, img_shape[0])
        ### TO-DO: Calc both polynomials using ploty, left_fit and right_fit ###
        self.left_fitx = self.left_fit[0]*self.ploty**2 + self.left_fit[1]*self.ploty + self.left_fit[2]
        self.right_fitx = self.right_fit[0]*self.ploty**2 + self.right_fit[1]*self.ploty + self.right_fit[2]

        #return left_fitx, right_fitx, left_fit, right_fit

    def search_around_poly(self, binary_warped):
        # HYPERPARAMETER
        # Choose the width of the margin around the previous polynomial to search
        # The quiz grader expects 100 here, but feel free to tune on your own!
        margin = 100

        # Grab activated pixels
        nonzero = binary_warped.nonzero()
        nonzeroy = np.array(nonzero[0])
        nonzerox = np.array(nonzero[1])

        ### TO-DO: Set the area of search based on activated x-values ###
        ### within the +/- margin of our polynomial function ###s
        left_lane_inds = ((nonzerox > (self.left_fit[0]*(nonzeroy**2) + self.left_fit[1]*nonzeroy + 
                        self.left_fit[2] - margin)) & (nonzerox < (self.left_fit[0]*(nonzeroy**2) + 
                        self.left_fit[1]*nonzeroy + self.left_fit[2] + margin)))
        right_lane_inds = ((nonzerox > (self.right_fit[0]*(nonzeroy**2) + self.right_fit[1]*nonzeroy + 
                        self.right_fit[2] - margin)) & (nonzerox < (self.right_fit[0]*(nonzeroy**2) + 
                        self.right_fit[1]*nonzeroy + self.right_fit[2] + margin)))

        # Again, extract left and right line pixel positions  --> index list
        leftx = nonzerox[left_lane_inds]
        lefty = nonzeroy[left_lane_inds] 
        rightx = nonzerox[right_lane_inds]
        righty = nonzeroy[right_lane_inds]

        # Fit new polynomials
        self.fit_poly(leftx, lefty, rightx, righty)
        #return left_fit, right_fit, left_fitx, right_fitx
 
        # *** Advance Search**********
    def find_lane_pixels(self, binary_warped):
        # Take a histogram of the bottom half of the image
        histogram = np.sum(binary_warped[binary_warped.shape[0]//2:,:], axis=0)
        # Find the peak of the left and right halves of the histogram
        # These will be the starting point for the left and right lines
        midpoint = np.int(histogram.shape[0]//2)
        leftx_base = np.argmax(histogram[:midpoint])
        rightx_base = np.argmax(histogram[midpoint:]) + midpoint

        # HYPERPARAMETERS
        # Choose the number of sliding windows
        nwindows = 12#9
        # Set the width of the windows +/- margin
        margin = 140#100
        # Set minimum number of pixels found to recenter window
        minpix = 30#50

        # Set height of windows - based on nwindows above and image shape
        window_height = np.int(binary_warped.shape[0]//nwindows)
        # Identify the x and y positions of all nonzero pixels in the image
        nonzero = binary_warped.nonzero()
        nonzeroy = np.array(nonzero[0])
        nonzerox = np.array(nonzero[1])
        # Current positions to be updated later for each window in nwindows
        leftx_current = leftx_base
        rightx_current = rightx_base

        # Create empty lists to receive left and right lane pixel indices
        left_lane_inds = []
        right_lane_inds = []

        # Step through the windows one by one
        for window in range(nwindows):
            # Identify window boundaries in x and y (and right and left)
            win_y_low = binary_warped.shape[0] - (window+1)*window_height
            win_y_high = binary_warped.shape[0] - window*window_height
            win_xleft_low = leftx_current - margin
            win_xleft_high = leftx_current + margin
            win_xright_low = rightx_current - margin
            win_xright_high = rightx_current + margin

            # Identify the nonzero pixels in x and y within the window #
            good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
            (nonzerox >= win_xleft_low) &  (nonzerox < win_xleft_high)).nonzero()[0]
            good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
            (nonzerox >= win_xright_low) &  (nonzerox < win_xright_high)).nonzero()[0]

            # Append these indices to the lists
            left_lane_inds.append(good_left_inds)
            right_lane_inds.append(good_right_inds)

            # If you found > minpix pixels, recenter next window on their mean position
            if len(good_left_inds) > minpix:
                leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
            if len(good_right_inds) > minpix:        
                rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

        # Concatenate the arrays of indices (previously was a list of lists of pixels)
        try:
            left_lane_inds = np.concatenate(left_lane_inds)
            right_lane_inds = np.concatenate(right_lane_inds)
        except ValueError:
            # Avoids an error if the above is not implemented fully
            pass

        # Extract left and right line pixel positions
        leftx = nonzerox[left_lane_inds]
        lefty = nonzeroy[left_lane_inds] 
        rightx = nonzerox[right_lane_inds]
        righty = nonzeroy[right_lane_inds]

        return leftx, lefty, rightx, righty

    def fit_polynomial(self, binary_warped):
        # Find our lane pixels first
        leftx, lefty, rightx, righty= self.find_lane_pixels(binary_warped)

        # Fit a second order polynomial to each using `np.polyfit`
        self.left_fit = np.polyfit(lefty, leftx, 2)
        self.right_fit = np.polyfit(righty, rightx, 2)

        # Generate x and y values for plotting
        #ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
        try:
            self.left_fitx = self.left_fit[0]*self.ploty**2 + self.left_fit[1]*self.ploty + self.left_fit[2]
            self.right_fitx = self.right_fit[0]*self.ploty**2 + self.right_fit[1]*self.ploty + self.right_fit[2]
        except TypeError:
            # Avoids an error if `left` and `right_fit` are still none or incorrect
            print('The function failed to fit a line!')
            self.left_fitx = 1*self.ploty**2 + 1*self.ploty
            self.right_fitx = 1*self.ploty**2 + 1*self.ploty

        #return left_fit, right_fit, left_fitx, right_fitx
           
            
        # bird_eye_view transformation
    def bird_eye_view_transformation(self, img):
        # Select area
        img_size= (img.shape[1], img.shape[0])
        # use cv2.warpPerspective() to warp your image to a top-down view
        warped = cv2.warpPerspective(img, self.perspective_M, img_size, flags=cv2.INTER_LINEAR)
        return warped
    
        # bird_eye_view transformation
    def bird_eye_view_matrix(self, img):
        # Select area
        img_size= (img.shape[1], img.shape[0])
        # 4 Points of the Trapezoid
        left_bottom = [210, img_size[1]]     #[201]
        right_bottom = [1105, img_size[1]]
        left_top = [595, 450]    #[598, 450]
        right_top = [687, 450]  
        #left_top = [568, 470] #[595, 450]    #[598, 450]
        #right_top = [717, 470]
        
        # 4 source points src = np.float32([[,],[,],[,],[,]])
        src= np.float32([left_top, right_top, right_bottom, left_bottom])
        # 4 destination points dst = np.float32([[,],[,],[,],[,]])
        offset = 300
        dst = np.float32([[offset, 0], [img_size[0]-offset, 0],[img_size[0]-offset, img_size[1]],[offset, img_size[1]]])
        # get M, the transform matrix
        M = cv2.getPerspectiveTransform(src, dst)
        return M
    
    ######## Tunning Functions: Record extra frames ######################
    # record a video to display also the warped image, the thresholded image and the sanity check values for tunning purpose 
    def record_4view(self,img):
        # any process you want
        self.count += 1
        #undistort
        img_undist = cal_undistort(img, Line.mtx, Line.dist)
        #binary thershold
        img_binary = pipeline(img_undist)
        color_binary = pipeline_color(img_undist)

        #starting configuration
        if (not self.configurated):
             #bird-eye-view
            self.perspective_M = self.bird_eye_view_matrix(img_binary)
            self.Minv = np.linalg.inv(self.perspective_M)
            binary_warped = self.bird_eye_view_transformation(img_binary)
            
            self.img_size = (binary_warped.shape[1], binary_warped.shape[0])
            self.ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0])
            #ploty = np.linspace(0, img_shape[0]-1, img_shape[0])
            # variables
            offset = 300
            self.ym_per_pix = 50/self.img_size[1]
            self.xm_per_pix = 3.7/(self.img_size[0]-2*offset)
            self.configurated = True

        else:
            self.old_left_fitx = self.left_fitx
            self.old_right_fitx = self.right_fitx
            self.old_left_fit = self.left_fit
            self.old_right_fit = self.right_fit
            
            binary_warped = self.bird_eye_view_transformation(img_binary)
              
        #First Frame- Complete Search:        
        if (not self.detected):
            #print("First Frame")
            # Here left_fitx ad left_fit (also right) are calculated
            color_warped = self.record_fit_polynomial(binary_warped)
            #self.search_around_poly(binary_warped)
            self.detected = True
            left_curverad, right_curverad = self.measure_curvature_real()
            self.old_left_fitx = self.left_fitx
            self.old_right_fitx = self.right_fitx
            self.old_left_fit = self.left_fit
            self.old_right_fit = self.right_fit
            
            #if First Frame is wrong, ignore this Frame and use the next
            margin=150
            check, [var_radius, var_topbottom, var_middlebottom, var_bottom_right, var_bottom_left] = self.sanity_check(left_curverad, right_curverad, margin)
            if (not check):
                self.detected = False
                
                color_warped = self.display_sanity_check(color_warped, var_radius, var_topbottom, var_middlebottom,  var_bottom_right, var_bottom_left)
                result4 = self.display_4img(img, color_warped, img_binary, color_binary)
                
                return result4
            else:
                #invert transform
                color_warped = self.display_sanity_check(color_warped, var_radius, var_topbottom, var_middlebottom,  var_bottom_right, var_bottom_left)
                result= self.invert_transformation(binary_warped, img_undist)
                result = self.show_curverad_position(result, left_curverad, right_curverad)
                result4 = self.display_4img(result, color_warped, img_binary, color_binary)
                return result4
            
        #Complete Search  
        elif self.count_undetected > 5:
            color_warped = self.record_fit_polynomial(binary_warped)
            #print(self.left_fit)
            #print(self.right_fit)
            #self.search_around_poly(binary_warped)           
            left_curverad, right_curverad = self.measure_curvature_real()
            
            margin=150
            check, [var_radius, var_topbottom, var_middlebottom, var_bottom_right, var_bottom_left] = self.sanity_check(left_curverad, right_curverad, margin)
            #not good: use old values
            if (not check):
                self.left_fitx = self.old_left_fitx
                self.right_fitx = self.old_right_fitx
                self.left_fit = self.old_left_fit
                self.right_fit = self.old_right_fit
                left_curverad, right_curverad = self.measure_curvature_real()
                #print("Sanity Change!")
                self.count_undetected += 1
                
                result= self.invert_transformation(binary_warped, img_undist)
                cv2.putText(result, text='Sanity check not passed on Window Search!', org=(50,250),
                    fontFace= cv2.FONT_HERSHEY_SIMPLEX, fontScale=1, color=(0,255,0),
                    thickness=2, lineType=cv2.LINE_AA)
                
                #visualization
                color_warped = cv2.polylines(color_warped, np.array([np.transpose(np.vstack([self.left_fitx, self.ploty]))], np.int32), False, (255,255,255), 4)
                color_warped = cv2.polylines(color_warped, np.array([np.transpose(np.vstack([self.right_fitx, self.ploty]))], np.int32), False, (255,255,255), 4)
                
            #good: use this values
            else:
                self.count_undetected = 0
                result= self.invert_transformation(binary_warped, img_undist)
        
            #invert transform
            #result= self.invert_transformation(binary_warped, img_undist)
            color_warped = self.display_sanity_check(color_warped, var_radius, var_topbottom, var_middlebottom, var_bottom_right, var_bottom_left)
            result = self.show_curverad_position(result, left_curverad, right_curverad)
            result4 = self.display_4img(result, color_warped, img_binary, color_binary)
            return result4
        
        #Simple Search
        else:
           # print("X Frame")
            color_warped = self.record_search_around_poly(binary_warped)
            #print(self.left_fit)
            #print(self.right_fit)
            # Calculate the radius of curvature in meters for both lane lines
            left_curverad, right_curverad = self.measure_curvature_real()
        
            margin=10
            check, [var_radius, var_topbottom, var_middlebottom,  var_bottom_right, var_bottom_left] = self.sanity_check(left_curverad, right_curverad, margin)
            
            if (not check):
                self.left_fitx = self.old_left_fitx
                self.right_fitx = self.old_right_fitx
                self.left_fit = self.old_left_fit
                self.right_fit = self.old_right_fit
                left_curverad, right_curverad = self.measure_curvature_real()
                #print("Sanity Change!")
                self.count_undetected+= 1
                
                result= self.invert_transformation(binary_warped, img_undist)
                cv2.putText(result, text='Sanity check not passed!', org=(50,250),
                    fontFace= cv2.FONT_HERSHEY_SIMPLEX, fontScale=1, color=(0,255,0),
                    thickness=2, lineType=cv2.LINE_AA)
                
                #visualization:
                color_warped = cv2.polylines(color_warped, np.array([np.transpose(np.vstack([self.left_fitx, self.ploty]))], np.int32), False, (255,255,255), 4)
                color_warped = cv2.polylines(color_warped, np.array([np.transpose(np.vstack([self.right_fitx, self.ploty]))], np.int32), False, (255,255,255), 4)
                
            else:
                self.count_undetected = 0
                result= self.invert_transformation(binary_warped, img_undist)
        
            #invert transform
            #result= self.invert_transformation(binary_warped, img_undist)
            color_warped = self.display_sanity_check(color_warped, var_radius, var_topbottom, var_middlebottom, var_bottom_right, var_bottom_left)
            result = self.show_curverad_position(result, left_curverad, right_curverad)
            result4 = self.display_4img(result, color_warped, img_binary, color_binary)
            return result4
    
    # display sanity check parameters
    def display_sanity_check(self, color_warped, var_radius, var_topbottom, var_middlebottom, var_bottom_right, var_bottom_left):
        
        cv2.putText(color_warped, text= 'v_rd = ' + str(var_radius), org=(int(self.img_size[0]/2 - 150),self.img_size[1]-250),
                    fontFace= cv2.FONT_HERSHEY_SIMPLEX, fontScale=1, color=(0,255,0),
                    thickness=2, lineType=cv2.LINE_AA)
        cv2.putText(color_warped, text= 'v_tb = ' + str(var_topbottom), org=(int(self.img_size[0]/2 - 150),self.img_size[1]-200),
                    fontFace= cv2.FONT_HERSHEY_SIMPLEX, fontScale=1, color=(0,255,0),
                    thickness=2, lineType=cv2.LINE_AA)
        cv2.putText(color_warped, text= 'v_mb = ' + str(var_middlebottom), org=(int(self.img_size[0]/2 - 150),self.img_size[1]-150),
                    fontFace= cv2.FONT_HERSHEY_SIMPLEX, fontScale=1, color=(0,255,0),
                    thickness=2, lineType=cv2.LINE_AA)
        cv2.putText(color_warped, text= 'v_bl = ' + str(var_bottom_left), org=(int(self.img_size[0]/2 - 150),self.img_size[1]-100),
                    fontFace= cv2.FONT_HERSHEY_SIMPLEX, fontScale=1, color=(0,255,0),
                    thickness=2, lineType=cv2.LINE_AA)
        cv2.putText(color_warped, text= 'v_br = ' + str(var_bottom_right), org=(int(self.img_size[0]/2 - 150),self.img_size[1]-50),
                    fontFace= cv2.FONT_HERSHEY_SIMPLEX, fontScale=1, color=(0,255,0),
                    thickness=2, lineType=cv2.LINE_AA)
        return color_warped
        
    # display 4 images on one: format 1
    def display_4img(self,result, color_warped, img_binary, color_binary):
        
        color_binary_green = np.dstack(( np.zeros_like(img_binary), img_binary, np.zeros_like(img_binary))) * 255
        #color_warped = np.dstack(( np.zeros_like(binary_warped), binary_warped, np.zeros_like(binary_warped))) * 255
        numpy_horizontal1 = np.hstack((result, color_warped))
        numpy_horizontal2 = np.hstack((color_binary_green, color_binary))
        new_size = (result.shape[1],result.shape[0])
        
        numpy_vertical = np.vstack((numpy_horizontal1, numpy_horizontal2))
        output = cv2.resize(numpy_vertical, new_size, interpolation = cv2.INTER_AREA)
        return output
    
    # display 4 images on one: format 2
    def display_4img2(self,result, img_binary, binary_warped, color_binary):
        
        color_binary_green = np.dstack(( np.zeros_like(img_binary), img_binary, np.zeros_like(img_binary))) * 255
        color_warped = np.dstack(( np.zeros_like(binary_warped), binary_warped, np.zeros_like(binary_warped))) * 255
        numpy_horizontal1 = np.hstack((result, color_warped))
        numpy_horizontal2 = np.hstack((color_binary_green, color_binary))
        new_size = (result.shape[1],result.shape[0])
        
        numpy_vertical = np.vstack((numpy_horizontal1, numpy_horizontal2))
        output = cv2.resize(numpy_vertical, new_size, interpolation = cv2.INTER_AREA)
        return output
    
    # record only binary threshold images on a video
    def record_binary(self,img):
        # any process you want
        self.count += 1
        #undistort
        img_undist = cal_undistort(img, Line.mtx, Line.dist)
        #binary thershold
        img_binary = pipeline_color(img_undist)
        
        cv2.putText(img_binary, text='Frame = ' + str(self.count), org=(50,150),
                    fontFace= cv2.FONT_HERSHEY_SIMPLEX, fontScale=1, color=(255,0,0),
                    thickness=2, lineType=cv2.LINE_AA)
            
        return img_binary
    
    # record only warped binary threshold images on a video
    def record_warped(self,img):
                # any process you want
        self.count += 1
        #undistort
        img_undist = cal_undistort(img, Line.mtx, Line.dist)
        #binary thershold
        img_binary = pipeline(img_undist)
        
                #starting configuration
        if (not self.configurated):
             #bird-eye-view
            self.perspective_M = self.bird_eye_view_matrix(img_binary)
            self.Minv = np.linalg.inv(self.perspective_M)
            binary_warped = self.bird_eye_view_transformation(img_binary)
            self.configurated = True

        else:
            binary_warped = self.bird_eye_view_transformation(img_binary)
            
        color_warped = np.dstack(( np.zeros_like(binary_warped), binary_warped, np.zeros_like(binary_warped))) * 255
        
        cv2.putText(color_warped, text='Frame = ' + str(self.count), org=(50,150),
                    fontFace= cv2.FONT_HERSHEY_SIMPLEX, fontScale=1, color=(255,0,0),
                    thickness=2, lineType=cv2.LINE_AA)
        return color_warped
    
    #expand search_around_poly()
    def record_search_around_poly(self, binary_warped):
        # HYPERPARAMETER
        # Choose the width of the margin around the previous polynomial to search
        # The quiz grader expects 100 here, but feel free to tune on your own!
        margin = 140 #100

        # Grab activated pixels
        nonzero = binary_warped.nonzero()
        nonzeroy = np.array(nonzero[0])
        nonzerox = np.array(nonzero[1])

        ### TO-DO: Set the area of search based on activated x-values ###
        ### within the +/- margin of our polynomial function ###s
        left_lane_inds = ((nonzerox > (self.left_fit[0]*(nonzeroy**2) + self.left_fit[1]*nonzeroy + 
                        self.left_fit[2] - margin)) & (nonzerox < (self.left_fit[0]*(nonzeroy**2) + 
                        self.left_fit[1]*nonzeroy + self.left_fit[2] + margin)))
        right_lane_inds = ((nonzerox > (self.right_fit[0]*(nonzeroy**2) + self.right_fit[1]*nonzeroy + 
                        self.right_fit[2] - margin)) & (nonzerox < (self.right_fit[0]*(nonzeroy**2) + 
                        self.right_fit[1]*nonzeroy + self.right_fit[2] + margin)))

        # Again, extract left and right line pixel positions  --> index list
        leftx = nonzerox[left_lane_inds]
        lefty = nonzeroy[left_lane_inds] 
        rightx = nonzerox[right_lane_inds]
        righty = nonzeroy[right_lane_inds]

        # Fit new polynomials
        self.fit_poly(leftx, lefty, rightx, righty)
        
            ## Visualization ##
        # Create an image to draw on and an image to show the selection window
        out_img = np.dstack((binary_warped, binary_warped, binary_warped))*255
        window_img = np.zeros_like(out_img)  #return zeros array with the same shape
        # Color in left and right line pixels
        out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
        out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]

        # Generate a polygon to illustrate the search window area
        # And recast the x and y points into usable format for cv2.fillPoly()
        left_line_window1 = np.array([np.transpose(np.vstack([self.left_fitx-margin, self.ploty]))]) # vstack=Stack arrays in sequence vertically (row wise).
        left_line_window2 = np.array([np.flipud(np.transpose(np.vstack([self.left_fitx+margin, 
                                  self.ploty])))]) #numpy.flipud = Flip array in the up/down direction.
        left_line_pts = np.hstack((left_line_window1, left_line_window2)) #np.hstack=Stack arrays in sequence horizontally (column wise).
        #array should draw the perimeter to fill later. ([x1 y1],[x2 y2])

        right_line_window1 = np.array([np.transpose(np.vstack([self.right_fitx-margin, self.ploty]))])
        right_line_window2 = np.array([np.flipud(np.transpose(np.vstack([self.right_fitx+margin, 
                                  self.ploty])))])
        right_line_pts = np.hstack((right_line_window1, right_line_window2))

        # Draw the lane onto the warped blank image
        # cv2.fillPoly expects [[[ [375   0] .... [575   0] ]]]
        cv2.fillPoly(window_img, np.int_([left_line_pts]), (0,255, 0))
        cv2.fillPoly(window_img, np.int_([right_line_pts]), (0,255, 0))
        result = cv2.addWeighted(out_img, 1, window_img, 0.3, 0)

        # Plot the polynomial lines onto the image
        #plt.plot(self.left_fitx, self.ploty, color='yellow')
        #plt.plot(self.right_fitx, self.ploty, color='yellow')
        result = cv2.polylines(result, np.array([np.transpose(np.vstack([self.left_fitx, self.ploty]))], np.int32), False, (0,255,0), 4)
        result = cv2.polylines(result, np.array([np.transpose(np.vstack([self.right_fitx, self.ploty]))], np.int32), False, (0,255,0), 4)
        #cv2.fillPoly(result, np.int_([left_line_pts]), (0,255, 0))
        
        return result
    
    ##expand fit_polynomial()
    def record_fit_polynomial(self, binary_warped):
        # Find our lane pixels first
        leftx, lefty, rightx, righty= self.find_lane_pixels(binary_warped)
        out_img = np.dstack((binary_warped, binary_warped, binary_warped))*255

        # Fit a second order polynomial to each using `np.polyfit`
        self.left_fit = np.polyfit(lefty, leftx, 2)
        self.right_fit = np.polyfit(righty, rightx, 2)

        # Generate x and y values for plotting
        #ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
        try:
            self.left_fitx = self.left_fit[0]*self.ploty**2 + self.left_fit[1]*self.ploty + self.left_fit[2]
            self.right_fitx = self.right_fit[0]*self.ploty**2 + self.right_fit[1]*self.ploty + self.right_fit[2]
        except TypeError:
            # Avoids an error if `left` and `right_fit` are still none or incorrect
            print('The function failed to fit a line!')
            self.left_fitx = 1*self.ploty**2 + 1*self.ploty
            self.right_fitx = 1*self.ploty**2 + 1*self.ploty

        ## Visualization ##
        # Colors in the left and right lane regions
        out_img[lefty, leftx] = [255, 0, 0]
        out_img[righty, rightx] = [0, 0, 255]

        # Plots the left and right polynomials on the lane lines
        result = cv2.polylines(out_img, np.array([np.transpose(np.vstack([self.left_fitx, self.ploty]))], np.int32), False, (0,255,0), 4)
        result = cv2.polylines(result, np.array([np.transpose(np.vstack([self.right_fitx, self.ploty]))], np.int32), False, (0,255,0), 4)
        #plt.plot(self.left_fitx, self.ploty, color='yellow')
        #plt.plot(self.right_fitx, self.ploty, color='yellow')
        #return left_fit, right_fit, left_fitx, right_fitx
        return result
    

In [10]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [10]:
video_output = 'output_videos/project_video_output_F1.mp4'

#3 mit thr 170
myline = Line() 
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4").subclip(0,5)
clip = VideoFileClip("project_video.mp4")
output_clip = clip.fl_image(myline) #NOTE: this function expects color images!!
%time output_clip.write_videofile(video_output, audio=False)

[MoviePy] >>>> Building video output_videos/project_video_output_F1.mp4
[MoviePy] Writing video output_videos/project_video_output_F1.mp4


100%|█████████████████████████████████████████████████████████████████████████████▉| 1260/1261 [03:13<00:00,  5.67it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: output_videos/project_video_output_F1.mp4 

Wall time: 3min 15s


In [3]:
video_output = 'output_videos/project_video_output_F1.mp4'

HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(video_output))

# Ploting Videos with Full Information

In [12]:
video_output2 = 'output_videos/project_video_output_4view.mp4'

#3 mit thr 170
myline = Line() 
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4").subclip(0,5)
clip1 = VideoFileClip("project_video.mp4")
output_clip1 = clip1.fl_image(myline.record_4view) #NOTE: this function expects color images!!
%time output_clip1.write_videofile(video_output2, audio=False)

[MoviePy] >>>> Building video output_videos/project_video_output_4view.mp4
[MoviePy] Writing video output_videos/project_video_output_4view.mp4


100%|█████████████████████████████████████████████████████████████████████████████▉| 1260/1261 [05:06<00:00,  3.19it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: output_videos/project_video_output_4view.mp4 

Wall time: 5min 8s


In [4]:
video_output2 = 'output_videos/project_video_output_4view_short.mp4'

HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(video_output2))

In [14]:
video_output_challenge = 'output_videos/challenge_video_output_4view.mp4'

myline_challenge = Line() 
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4").subclip(0,5)
clip2 = VideoFileClip("challenge_video.mp4")
output_clip2 = clip2.fl_image(myline_challenge.record_4view) #NOTE: this function expects color images!!
%time output_clip2.write_videofile(video_output_challenge, audio=False)

[MoviePy] >>>> Building video output_videos/challenge_video_output_4view.mp4
[MoviePy] Writing video output_videos/challenge_video_output_4view.mp4


100%|████████████████████████████████████████████████████████████████████████████████| 485/485 [01:51<00:00,  4.30it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: output_videos/challenge_video_output_4view.mp4 

Wall time: 1min 53s


In [12]:
video_output_challenge = 'output_videos/challenge_video_output_4view.mp4'

HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(video_output_challenge))

In [16]:
video_output_hchallenge = 'output_videos/hchallenge_video_output_4view1.mp4'

myline_hchallenge = Line() 
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4").subclip(0,5)
clip3 = VideoFileClip("harder_challenge_video.mp4")
output_clip3 = clip3.fl_image(myline_hchallenge.record_4view) #NOTE: this function expects color images!!
%time output_clip3.write_videofile(video_output_hchallenge, audio=False)

[MoviePy] >>>> Building video output_videos/hchallenge_video_output_4view1.mp4
[MoviePy] Writing video output_videos/hchallenge_video_output_4view1.mp4


100%|█████████████████████████████████████████████████████████████████████████████▉| 1199/1200 [06:10<00:00,  3.44it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: output_videos/hchallenge_video_output_4view1.mp4 

Wall time: 6min 13s


In [22]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(video_output_hchallenge))

In [23]:
frame_number = 1
clip10 = VideoFileClip("output_videos/hchallenge_video_output_4view1.mp4")
clip10.save_frame('output_images/tunning_hchallenge_'+ str(frame_number) +'.jpg', 00.000)

In [15]:
frame_number = 1
clip2 = VideoFileClip("harder_challenge_video.mp4")
clip2.save_frame('output_images/undist_frameh'+ str(frame_number) +'.jpg', 00.000)